<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Import-modules" data-toc-modified-id="Import-modules-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import modules</a></span></li><li><span><a href="#Get-dataverse-info" data-toc-modified-id="Get-dataverse-info-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get dataverse info</a></span></li><li><span><a href="#Get-dataset-info" data-toc-modified-id="Get-dataset-info-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Get dataset info</a></span></li></ul></li></ul></div>

## Plan

For chosen dataverse:
    - Show dataverse info:
        - Show number of subdataverses, if any
        - Show whether or not the (sub)dataverse has a description or tagline
        - List metadatablocks of each (sub)dataverse
        - List the facets set for each (sub)dataverse
        - Verify that contact email address is valid for each (sub)dataverse
        (see https://medium.com/@arjunsinghy96/verify-emails-over-socks-proxy-using-python-5589cb75c405
        and https://github.com/Gardener-gg/email-verifier)
    - Show dataset info
        - Show number of datasets
        - Show date of first published dataset
        - Show date of most recently published or updated dataset
        - Show average age of datasets
        - Show average number of dataset versions
        - Metadata (of latest published version of each dataset):
            - Show average number of characters in the dataset descriptions
                - List datasets with fewer than a certain number of characters in their descriptions
            - List datasets with CC0 or Terms of Use metadata
                - Versus number of datasets with no CC0 or TOU metadata
                - List datasets with no CC0 or TOU metadata
            - Show number of files that have no description metadata
                - If a certain percentage of datasets have 1 or more files with no descriptions, list those datasets
            - Related publication metadata
                - Show number of datasets with related publication metadata
                - Show number of datasets with incomplete related publication metadata
                    - List datasets with incomplete related publication metadata
                - Show number of datasets with no related publication metadata
                    - List datasets with no related publication metadata
            - Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse
        - Data
            - List count of each unique file format
            - Show number of datasets with no files
                - List datasets that have no files
            - Show number of datasets with 1 or more uningested tabular files
                - List datasets that contain 1 or more uningested tabular files
            - Show number of datasets with 1 or more restricted files
        - Contact emails
            - Get number of datasets that have a contact email address that's different email from the dataverse contact email address
            - Get unique list of contact email addresses and check if they're valid
            - Show any datasets that have no valid email addresses

## Code

### Import modules

In [1]:
import numpy as np
import pandas as pd
import requests

def improved_get(_dict, path, default=None):
    for key in path.split('.'):
        try:
            _dict = _dict[key]
        except KeyError:
            return default
    return _dict

### Get dataverse info

In [54]:
# Get dataverse alias from user - return error if there's no alias or if alias is the Root dataverse
# dataverseAlias = 'covid-rt'
# server = 'https://demo.dataverse.org'
# dataverseAlias = 'test777'
server = 'https://dataverse.harvard.edu'
dataverseAlias = 'CISI'

In [55]:
# Get info from that dataverse: whether or not the dataverse has a description and/or tagline, metadatablocks enabled, facets enabled, validate contact email
dataverseInfoApi = '%s/api/dataverses/%s' % (server, dataverseAlias)
response = requests.get(dataverseInfoApi)
dataverseMetadata = response.json()

In [57]:
if dataverseMetadata['status'] == 'ERROR':
    print('No dataverse found. Is the dataverse published on Harvard Dataverse?')
elif dataverseMetadata['status'] == 'OK':
    dataverseDescription = improved_get(dataverseMetadata, 'data.description')
    if dataverseDescription is not None:
        print('Dataverse has a description')
    else:
        print('Dataverse has no description')

    tagline = improved_get(dataverseMetadata, 'data.theme.tagline')
    if tagline is not None:
        print('Dataverse has a tagline')
    else:
        print('Dataverse has no tagline')

#     contactEmails = []
#     for contact in dataverseMetadata['data']['dataverseContacts']:
#         contactEmails.append(contact['contactEmail'])
#     print(contactEmails)

    dataverseFacetsApi = '%s/api/dataverses/%s/facets' % (server, dataverseAlias)
    response = requests.get(dataverseFacetsApi)
    dataverseFacets = response.json()
    facets = []
    for facet in dataverseFacets['data']:
        facets.append(facet)
    print('Number of search facets used: %s' % (len(facets)))    

    metadatablocks = []
    dataverseMetadatablocksApi = '%s/api/dataverses/%s/metadatablocks' % (server, dataverseAlias)
    response = requests.get(dataverseMetadatablocksApi)
    dataverseMetadatablocks = response.json()
    for metadatablock in dataverseMetadatablocks['data']:
        metadatablock = metadatablock['name']
        metadatablocks.append(metadatablock)
    print('Number of metadatablocks enabled (in addition to Citation): %s' % (len(metadatablocks) - 1))

Dataverse has a description
Dataverse has a tagline
Number of search facets used: 3
Number of metadatablocks enabled (in addition to Citation): 0


### Get dataset info

In [58]:
# Get PIDs of all published datasets and their files in that dataverse
    # Use Get Contents API to get dataset PIDs and publication date
    # (ex: https://demo.dataverse.org/api/dataverses/sefsef/contents)
getContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseAlias)
response = requests.get(getContentsApi)
dataverseContents = response.json()

In [59]:
datasetPIDs = []
for item in dataverseContents['data']:
    if item['type'] == 'dataset':
        datasetPID = item['persistentUrl'].replace('https://doi.org/', 'doi:')
        datasetPIDs.append(datasetPID)
print('Number of datasets: %s' % (len(datasetPIDs)))

Number of datasets: 20


Create a dataframe for dataset info: date of publication, the release date of the latest version, number of versions

_Getting this info can be slow, e.g. for ~375 datasets it might take 45 min_

In [68]:
uningestedFileTypes = ['application/x-rlang-transport', 'application/x-stata-13', 'application/x-spss-por',
                      'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'text/csv', 'text/tsv',
                      'application/x-spss-sav', 'text/comma-separated-values', 'application/x-stata',
                      'application/x-stata-14']

rowList = []
count = 0
for datasetPID in datasetPIDs:
    getAllVersionsApi = '%s/api/datasets/:persistentId/versions?persistentId=%s' % (server, datasetPID)
    response = requests.get(getAllVersionsApi)
    datasetVersions = response.json()
    
    # Get only datasets with metadata (exclude responses with no values in 'data' key, e.g. deaccessioned datasets)
    if datasetVersions['status'] == 'OK' and len(datasetVersions['data']) > 0:

        # Get index location of first dataset version
        firstVersion = len(datasetVersions['data']) - 1

        # Get length of description text
        descriptionLength = 0
        for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'dsDescription':
                for i in field['value']:
                    descriptionLength = descriptionLength + len(i['dsDescriptionValue']['value'])

        # See whether CC0 or Terms of Use metadata exists
        license = datasetVersions['data'][0]['license']
        try:
            termsOfUse = datasetVersions['data'][0]['termsOfUse']
            termsOfUse = True
        except KeyError:
            termsOfUse = False

        if license != 'CC0' and termsOfUse == False:
            termsExist = False
        else:
            termsExist = True
        
        '''
        - Related publication metadata
            - Show number of datasets with related publication metadata
            - Show number of datasets with incomplete related publication metadata
                - List datasets with incomplete related publication metadata
            - Show number of datasets with no related publication metadata
                - List datasets with no related publication metadata
        '''
        # Get info about related publication metadata
        
        
        # Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse

        # Get number of files
        numberOfFiles = len(datasetVersions['data'][0]['files'])

        # Get file info
        noFileDescriptionCount = 0
        contentType = []
        uningestedTabFilesCount = 0
        restrictedFilesCount = 0
        for file in datasetVersions['data'][0]['files']:            
            if 'description' in file:
                noFileDescriptionCount = noFileDescriptionCount
            else:
                noFileDescriptionCount += 1
            contentType.append(file['dataFile']['contentType'])
            if file['restricted'] == True:
                restrictedFilesCount += 1
            if file['dataFile']['contentType'] in uningestedFileTypes:
                uningestedTabFilesCount += 1

        if len(contentType) == 0:
            uniqueContentTypes = 'NA'
        else:
            uniqueContentTypes = repr(set(contentType)).lstrip('{').rstrip('}')

        # Create dictionary
        newRow = {'datasetPID': datasetPID,
                  'numberOfVersions': len(datasetVersions['data']),
                  'publicationDate': datasetVersions['data'][firstVersion]['releaseTime'],
                  'latestReleaseDate': datasetVersions['data'][0]['releaseTime'],
                  'descriptionLenth': descriptionLength,
                  'termsExist': termsExist,
                  'numberOfFiles': numberOfFiles,
                  'noFileDescriptionCount': noFileDescriptionCount,
                  'uniqueContentTypes': uniqueContentTypes,
                  'uningestedTabFilesCount': uningestedTabFilesCount,
                  'restrictedFilesCount': restrictedFilesCount
                 }
        rowList.append(dict(newRow))
        count += 1
        print('%s of %s (%s)' % (count, len(datasetPIDs), datasetPID))
        
if datasetPIDs != count:
    print('The metadata of %s dataset(s) could not be retrieved' % (len(datasetPIDs) - count))


1 of 20 (doi:10.7910/DVN/NIESKT)
2 of 20 (doi:10.7910/DVN/LAGXUO)
3 of 20 (doi:10.7910/DVN/ZEBINZ)
4 of 20 (doi:10.7910/DVN/Y4WQ10)
5 of 20 (doi:10.7910/DVN/1MVDY7)
6 of 20 (doi:10.7910/DVN/SZBDIE)
7 of 20 (doi:10.7910/DVN/YEH8PF)
8 of 20 (doi:10.7910/DVN/NSE8BB)
9 of 20 (doi:10.7910/DVN/UOACQP)
10 of 20 (doi:10.7910/DVN/MIC0Z2)
11 of 20 (doi:10.7910/DVN/JRFVTJ)
12 of 20 (doi:10.7910/DVN/W6T1YV)
13 of 20 (doi:10.7910/DVN/U6GYPN)
14 of 20 (doi:10.7910/DVN/IXQ1BU)
15 of 20 (doi:10.7910/DVN/ATJ3NW)
16 of 20 (doi:10.7910/DVN/VSAIVR)
17 of 20 (doi:10.7910/DVN/KDVQIW)
18 of 20 (doi:10.7910/DVN/SXXZZS)
19 of 20 (doi:10.7910/DVN/VCJKH7)
The metadata of 1 dataset(s) could not be retrieved:


In [65]:
df = pd.DataFrame(rowList)               
df

,datasetPID,numberOfVersions,publicationDate,latestReleaseDate,descriptionLenth,termsExist,numberOfFiles,noFileDescriptionCount,uniqueContentTypes,uningestedTabFilesCount,restrictedFilesCount
0,doi:10.7910/DVN/NIESKT,2,2020-10-07T10:19:44Z,2020-10-26T14:47:12Z,158,True,3,0,"'application/pdf', 'application/x-stata-syntax...",1,0
1,doi:10.7910/DVN/LAGXUO,1,2016-09-18T16:00:00Z,2016-09-18T16:00:00Z,160,True,2,0,"'application/vnd.ms-excel', 'application/vnd.o...",0,0
2,doi:10.7910/DVN/ZEBINZ,1,2016-10-19T13:00:44Z,2016-10-19T13:00:44Z,241,True,67,67,'application/vnd.openxmlformats-officedocument...,0,0
3,doi:10.7910/DVN/Y4WQ10,1,2016-12-24T14:47:52Z,2016-12-24T14:47:52Z,105,True,3,3,'application/vnd.openxmlformats-officedocument...,1,0
4,doi:10.7910/DVN/1MVDY7,1,2018-04-29T19:50:33Z,2018-04-29T19:50:33Z,192,True,3,2,"'application/vnd.ms-excel', 'application/vnd.o...",0,0
5,doi:10.7910/DVN/SZBDIE,1,2017-05-05T12:40:29Z,2017-05-05T12:40:29Z,187,True,1,1,'text/tab-separated-values',0,0
6,doi:10.7910/DVN/YEH8PF,1,2016-07-05T22:02:56Z,2016-07-05T22:02:56Z,101,True,10,0,"'application/vnd.ms-excel', 'application/vnd.o...",4,0
7,doi:10.7910/DVN/NSE8BB,1,2018-01-03T18:52:46Z,2018-01-03T18:52:46Z,152,True,1,1,'application/vnd.openxmlformats-officedocument...,1,0
8,doi:10.7910/DVN/UOACQP,1,2016-07-20T10:36:47Z,2016-07-20T10:36:47Z,121,True,2,2,'application/vnd.openxmlformats-officedocument...,1,0
9,doi:10.7910/DVN/MIC0Z2,1,2018-06-22T17:01:32Z,2018-06-22T17:01:32Z,97,True,5,1,'application/vnd.openxmlformats-officedocument...,2,0


In [69]:
df.to_csv('sample.csv')

In [ ]:
# Show average number of characters in the dataset descriptions
for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
    if field['typeName'] == 'dsDescription':
        print(field[])
#     descriptionLength = len(field['value'])

In [ ]:
# List datasets with fewer than a certain number of characters in their descriptions